### Imports

In [1]:
import pandas as pd
from decouple import config
import requests
import csv

### First Steps

Start with a small test sample of books, scanned using a barcode scanner

In [6]:
searchTerm = "9319256000988"

In [3]:
# pulling in the authentication information from the env variable
apiKey = config('KEY')
apiUser = config('USER')

In [7]:
# making an initial request
payload = {'key' : apiKey}
r = requests.get(f'https://www.googleapis.com/books/v1/volumes?q=isbn:{searchTerm}', params = payload)
r.json()

{'kind': 'books#volumes', 'totalItems': 0}

In [5]:
r.json()

{'kind': 'books#volumes',
 'totalItems': 1,
 'items': [{'kind': 'books#volume',
   'id': 'ektGSwAACAAJ',
   'etag': 'vKRBLsE6j/8',
   'selfLink': 'https://www.googleapis.com/books/v1/volumes/ektGSwAACAAJ',
   'volumeInfo': {'title': 'How Language Works',
    'authors': ['David Crystal', 'Steven Pinker'],
    'publishedDate': '2008',
    'description': "David Crystal's How Language Works is a fascinating tour through the world of language from one of today's most renowned experts. It ranges over everything from how children learn to read to what makes words rude or polite, from eyebrow flashes to whistling languages. Unlocking the secrets of communication in an accessible, entertaining way, this exhilarating book sheds light on the endless mysteries of the language we speak, write and read every day.",
    'industryIdentifiers': [{'type': 'ISBN_10', 'identifier': '0141037369'},
     {'type': 'ISBN_13', 'identifier': '9780141037363'}],
    'readingModes': {'text': False, 'image': False},

In [6]:
r.json()['items'][0]['volumeInfo']['title']

'How Language Works'

In [52]:
def getDeets(isbn):
    row = {}
    row['ISBN'] = isbn
    payload = {'key' : apiKey}
    r = requests.get(f'https://www.googleapis.com/books/v1/volumes?q=isbn:{isbn}', params = payload)
    info = r.json()['items'][0]['volumeInfo']
    try:
        row['title'] = info['title']
        if 'subtitle' in info:
            row['subtitle'] = info['subtitle']
        else:
            row['subtitle'] = 'NA'
        row['authors'] = info['authors']
        row['pageCount'] = info['pageCount']
        row['categories'] = info['categories']
        row['averageRating'] = info['averageRating']
        row['ratingsCount'] =  info['ratingsCount']
        row['publishedDate'] = info['publishedDate']
        row['publisher'] = info['publisher']
        row['description'] = info['description']
    except Exception as e:
        print(f"Problem with gathering info: \n{e}" )
        
    for k, v in row.items():
        print(f"{k} : {v}")
    print("\n")
    
    # now print to csv
    try:
        with open('output.csv', "a", newline='') as csvFile:
            writer = csv.writer(csvFile, delimiter=',')
            writer.writerow([row['ISBN'], 
                             row['title'], 
                             row['subtitle'], 
                             row['authors'], 
                             row['pageCount'], 
                             row['categories'], 
                             row['averageRating'], 
                             row['ratingsCount'],
                             row['publishedDate'], 
                             row['publisher'], 
                             row['description']])
    except Exception as e:
        print(f"Problem with writing to file: \n{e}" )

In [53]:
for eachISBN in df['ISBN'].to_list()[1:]:
    getDeets(eachISBN)

ISBN : 9780151005574
title : Looking for Spinoza
subtitle : Joy, Sorrow, and the Feeling Brain
authors : ['Antonio R. Damasio']
pageCount : 355
categories : ['Philosophy']
averageRating : 3.5
ratingsCount : 6
publishedDate : 2003
publisher : Houghton Mifflin
description : Investigates the cerebral mechanisms behind emotions and feelings to explain the role between emotion, survival, and cultural accomplishment.


ISBN : 9781473620919
title : Thing Explainer
subtitle : Complicated Stuff in Simple Words
authors : ['Randall Munroe']
pageCount : 61
categories : ['Humor']
averageRating : 3.5
ratingsCount : 8
publishedDate : 2015-11-19
publisher : John Murray
description : From the No. 1 bestselling author of What If? - the man who created xkcd and explained the laws of science with cartoons - comes a series of brilliantly simple diagrams ('blueprints' if you want to be complicated about it) that show how important things work: from the nuclear bomb to the biro. It's good to know what the pa